# Import

In [1]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/MyDrive/TTKS 20203/VIB/Track 1 Datarathon/'
%cd $path
!ls

Mounted at /content/drive
/content/drive/MyDrive/TTKS 20203/VIB/Track 1 Datarathon
'0.Data VIB Hackathon Guidline.xlsx'   5.Data_Lending.csv   log.csv
 1.Data_Customer.csv		       6.Data_Card.csv	    logs.csv
 2.Data_MyVIB_Transaction.csv	       feature.csv	    members.csv
 3.Data_MyVIB_Activity.csv	       feature_new.csv	    tran.csv
 3.Data_MyVIB_Activity_New	       features.txt	    trans.csv
 4.Data_Deposit.csv		       label_times.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm

pd.options.mode.chained_assignment = None
sns.set()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
label_times = pd.read_csv(path + 'label_times.csv',
                        parse_dates=['CUTOFF_TIME'],
                        infer_datetime_format = True)
cutoff_times = label_times.drop(label_times.columns[[0]], axis = 1)
cutoff_times.head()

,CUSTOMER_NUMBER,CUTOFF_TIME,LABEL,DAYS_TO_CHURN,CHURN_DATE
0,0,2019-09-01,0.0,74.0,NaN
1,0,2019-10-01,1.0,44.0,2019-11-14
2,0,2019-11-01,0.0,13.0,NaN
3,0,2019-12-01,0.0,NaN,NaN
4,0,2020-01-01,0.0,NaN,NaN


In [4]:
members = pd.read_csv(path + 'members.csv')
members.head()

,CUSTOMER_NUMBER,CLIENT_SEX,STAFF_VIB,EB_REGISTER_CHANNEL,SMS,VERIFY_METHOD,AGE_AT_CREATE,CREATE_TO_IB_REGISTER
0,639362,"[0, 1]","[1, 0]","[0, 1, 0, 0]","[0, 1]","[0, 0, 1, 0]",8514.0,1.0
1,452440,"[0, 1]","[1, 0]","[0, 1, 0, 0]","[0, 1]","[0, 1, 0, 0]",7034.0,1.0
2,326290,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[0, 1]","[0, 0, 1, 0]",8368.0,1.0
3,20802,"[0, 1]","[1, 0]","[0, 1, 0, 0]","[0, 1]","[0, 1, 0, 0]",9959.0,1.0
4,114244,"[0, 1]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 0, 1, 0]",16602.0,1.0


In [6]:
def sum_vec(col):
    mylist = []

    for arr in col:
        mylist.append(eval(arr))

    return [sum(x) for x in zip(*mylist)]

In [7]:
def cal_previous_month(time):
    if time.month == 1:
        previous_month = pd.datetime(time.year-1, 12, 1)
    else:
        previous_month = pd.datetime(time.year, time.month - 1, 1)

    return previous_month

# Transaction

In [8]:
trans = pd.read_csv(path + 'trans.csv')
cols = ['TRANS_FREQUENCY','AVG_AMOUNT_TRANS_DAY','AVG_AMOUNT_TRANS']
trans.drop(cols, axis=1, inplace=True)
trans[trans['CUSTOMER_NUMBER']==0]

,TRANS_DATE,CUSTOMER_NUMBER,SUM_TRANS_NO,SUM_TRANS_AMOUNT,TRANS_PER_DAY,TRANS_LV1_VEC,TRANS_LV2_VEC
416969,2019-09-26,0,1,99800000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
502968,2019-10-14,0,1,2000000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
770690,2019-11-29,0,2,74000000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
777656,2019-11-30,0,1,2000000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
784483,2019-12-01,0,1,200000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
790576,2019-12-02,0,1,38000000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
805740,2019-12-04,0,2,3819000,2,"[2, 0, 0]","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
931707,2019-12-21,0,1,10000000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
988448,2019-12-29,0,1,20000000,1,"[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [21]:
def trans_cutoff(id):
    trans_cutoff = cutoff_times[cutoff_times['CUSTOMER_NUMBER']==id]
    timestamp = list(trans_cutoff['CUTOFF_TIME'])

    for time in timestamp:
        previous_month = cal_previous_month(time)

        TRAN = trans[(trans['CUSTOMER_NUMBER']==id) & (pd.to_datetime(trans['TRANS_DATE']) < time)]
        TRAN_PM = TRAN[pd.to_datetime(TRAN['TRANS_DATE']) >= previous_month]

        # Giao dịch gần nhất trước cutoff_time
        LAST_TRANS_DATE = pd.to_datetime(TRAN_PM['TRANS_DATE'].max())

        if pd.isnull(LAST_TRANS_DATE):
            trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'LAST_TRANS'] = np.nan
        else:
            trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'LAST_TRANS'] = (time - LAST_TRANS_DATE).days

        # Tổng số giao dịch trong tháng trước đó, trước cutoff time
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_NO_PM'] = TRAN_PM['SUM_TRANS_NO'].sum()
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_NO'] = TRAN['SUM_TRANS_NO'].sum()

        # Tổng số tiền giao dịch tháng trước đó, trước cutoff time
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_AMOUNT_PM'] = TRAN_PM['SUM_TRANS_AMOUNT'].sum()
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_AMOUNT'] = TRAN['SUM_TRANS_AMOUNT'].sum()

        # Số ngày giao dịch trong tháng trước đó, trước cutoff time
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_DATE_PM'] = len(TRAN_PM)
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_DATE'] = len(TRAN)

        # Tổng số khung giờ giao dịch trong tháng trước đó, trước cutof time
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_HOUR_PM'] = TRAN_PM['TRANS_PER_DAY'].sum()   
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'SUM_TRANS_HOUR'] = TRAN['TRANS_PER_DAY'].sum()   
        
        
        # Các loại giao dịch LV1 trước cutoff time, trong tháng trước đó
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'TRANS_LV1'] = str(sum_vec(TRAN['TRANS_LV1_VEC']))
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'TRANS_LV1_PM'] = str(sum_vec(TRAN_PM['TRANS_LV1_VEC']))

        # Các loại giao dịch LV2 trước cutoff time, trong tháng trước đó
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'TRANS_LV2'] = str(sum_vec(TRAN['TRANS_LV2_VEC']))
        trans_cutoff.loc[trans_cutoff['CUTOFF_TIME'] == time,'TRANS_LV2_PM'] = str(sum_vec(TRAN_PM['TRANS_LV2_VEC']))

    return trans_cutoff

TRANSACTIONS_0 = trans_cutoff(0)
TRANSACTIONS_0

,CUSTOMER_NUMBER,CUTOFF_TIME,LABEL,DAYS_TO_CHURN,CHURN_DATE,LAST_TRANS,SUM_TRANS_NO_PM,SUM_TRANS_NO,SUM_TRANS_AMOUNT_PM,SUM_TRANS_AMOUNT,SUM_TRANS_DATE_PM,SUM_TRANS_DATE,SUM_TRANS_HOUR_PM,SUM_TRANS_HOUR,TRANS_LV1,TRANS_LV1_PM,TRANS_LV2,TRANS_LV2_PM
0,0,2019-09-01,0.0,74.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[],[]
1,0,2019-10-01,1.0,44.0,2019-11-14,5.0,1.0,1.0,99800000.0,99800000.0,1.0,1.0,1.0,1.0,"[1, 0, 0]","[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,0,2019-11-01,0.0,13.0,NaN,18.0,1.0,2.0,2000000.0,101800000.0,1.0,2.0,1.0,2.0,"[2, 0, 0]","[1, 0, 0]","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,0,2019-12-01,0.0,NaN,NaN,1.0,3.0,5.0,76000000.0,177800000.0,2.0,4.0,2.0,4.0,"[4, 0, 0]","[2, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,0,2020-01-01,0.0,NaN,NaN,3.0,6.0,11.0,72019000.0,249819000.0,5.0,9.0,6.0,10.0,"[10, 0, 0]","[6, 0, 0]","[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# TRANSACTIONS = []
# for id in tqdm(INTERSECT_ID):
#     TRANSACTIONS.append(trans_cutoff(id))

# pd.concat(TRANSACTIONS)

# Activity

In [10]:
logs = pd.read_csv(path + 'logs.csv')
cols = ['ACTIVITY_FREQUENCY']
logs.drop(cols, axis=1, inplace=True)
logs[logs['CUSTOMER_NUMBER']==0]

,ACTIVITY_DATE,CUSTOMER_NUMBER,SUM_ACTIVITY_NO,ACTIVE_ON_APP_TIME,ACTIVITY_NAME_VEC
911266,2019-09-26,0,10,1,"[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
981027,2019-10-03,0,13,1,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1071095,2019-10-11,0,7,1,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1083092,2019-10-12,0,3,1,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1101113,2019-10-14,0,6,2,"[1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1699229,2019-11-29,0,34,4,"[2, 3, 3, 2, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1713606,2019-11-30,0,4,1,"[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1728370,2019-12-01,0,4,1,"[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1740740,2019-12-02,0,10,2,"[1, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1771667,2019-12-04,0,8,2,"[0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ..."


In [19]:
def logs_cutoff(id):
    logs_cutoff = cutoff_times[cutoff_times['CUSTOMER_NUMBER']==id]
    timestamp = list(logs_cutoff['CUTOFF_TIME'])

    for time in timestamp:
        previous_month = cal_previous_month(time)

        LOG = logs[(logs['CUSTOMER_NUMBER']==id) & (pd.to_datetime(logs['ACTIVITY_DATE']) < time)]
        LOG_PM = LOG[pd.to_datetime(LOG['ACTIVITY_DATE']) >= previous_month]

        # Hoạt gần nhất trước cutoff_time
        LAST_ACTIVITY_DATE = pd.to_datetime(LOG_PM['ACTIVITY_DATE'].max())

        if pd.isnull(LAST_ACTIVITY_DATE):
            logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'LAST_ACTIVITY'] = np.nan
        else:
            logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'LAST_ACTIVITY'] = (time - LAST_ACTIVITY_DATE).days

        # Tổng số hoạt động trong tháng trước đó, trước cutoff time
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'SUM_ACTIVITY_NO_PM'] = LOG_PM['SUM_ACTIVITY_NO'].sum()
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'SUM_ACTIVITY_NO'] = LOG['SUM_ACTIVITY_NO'].sum()

        # Tổng số lần dùng app trong tháng trước đó, trước cutoff time
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'ON_APP_PM'] = LOG_PM['ACTIVE_ON_APP_TIME'].sum()
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'ON_APP'] = LOG['ACTIVE_ON_APP_TIME'].sum()

        # Số ngày hoạt động trên app trong tháng trước đó, trước cutoff time
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'SUM_ON_APP_DATE_PM'] = len(LOG_PM)
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'SUM_ON_APP_DATE'] = len(LOG)

        # Các loại hoạt động trong tháng trước đó, trước cutoff time
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'ACTIVITY_NAME_PM'] = str(sum_vec(LOG_PM['ACTIVITY_NAME_VEC']))
        logs_cutoff.loc[logs_cutoff['CUTOFF_TIME'] == time,'ACTIVITY_NAME'] = str(sum_vec(LOG['ACTIVITY_NAME_VEC']))

    return logs_cutoff

Activities_0 = logs_cutoff(0)
Activities_0

,CUSTOMER_NUMBER,CUTOFF_TIME,LABEL,DAYS_TO_CHURN,CHURN_DATE,LAST_ACTIVITY,SUM_ACTIVITY_NO_PM,SUM_ACTIVITY_NO,ON_APP_PM,ON_APP,SUM_ON_APP_DATE_PM,SUM_ON_APP_DATE,ACTIVITY_NAME_PM,ACTIVITY_NAME
0,0,2019-09-01,0.0,74.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
1,0,2019-10-01,1.0,44.0,2019-11-14,5.0,10.0,10.0,1.0,1.0,1.0,1.0,"[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,0,2019-11-01,0.0,13.0,NaN,18.0,29.0,39.0,5.0,6.0,4.0,5.0,"[2, 4, 2, 2, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[3, 5, 3, 2, 5, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, ..."
3,0,2019-12-01,0.0,NaN,NaN,1.0,38.0,77.0,5.0,11.0,2.0,7.0,"[2, 4, 3, 2, 4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[5, 9, 6, 4, 9, 1, 0, 1, 4, 0, 0, 0, 0, 0, 0, ..."
4,0,2020-01-01,0.0,NaN,NaN,3.0,35.0,112.0,7.0,18.0,5.0,12.0,"[3, 7, 1, 0, 7, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, ...","[8, 16, 7, 4, 16, 1, 0, 1, 10, 0, 0, 0, 0, 0, ..."


# Concat

In [20]:
mem_cusid = list(members['CUSTOMER_NUMBER'].unique())
tran_cusid = list(trans['CUSTOMER_NUMBER'].unique())
log_cusid = list(logs['CUSTOMER_NUMBER'].unique())
cut_cusid = list(cutoff_times['CUSTOMER_NUMBER'].unique())
INTERSECT_ID = set.intersection(*map(set,[mem_cusid,tran_cusid,log_cusid,cut_cusid]))
len(INTERSECT_ID)

52112

In [22]:
Members_0 = members[members['CUSTOMER_NUMBER'] == 0]
Members_0

,CUSTOMER_NUMBER,CLIENT_SEX,STAFF_VIB,EB_REGISTER_CHANNEL,SMS,VERIFY_METHOD,AGE_AT_CREATE,CREATE_TO_IB_REGISTER
56736,0,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 1, 0, 0]",11301.0,2.0


In [28]:
# pd.concat(Members_0, TRANSACTIONS_0, Activities_0)
all_data = pd.merge(Activities_0, TRANSACTIONS_0, on=['CUSTOMER_NUMBER','CUTOFF_TIME','LABEL', 'DAYS_TO_CHURN','CHURN_DATE'])
all_data = pd.merge(Members_0, all_data, on=['CUSTOMER_NUMBER'])
all_data

,CUSTOMER_NUMBER,CLIENT_SEX,STAFF_VIB,EB_REGISTER_CHANNEL,SMS,VERIFY_METHOD,AGE_AT_CREATE,CREATE_TO_IB_REGISTER,CUTOFF_TIME,LABEL,DAYS_TO_CHURN,CHURN_DATE,LAST_ACTIVITY,SUM_ACTIVITY_NO_PM,SUM_ACTIVITY_NO,ON_APP_PM,ON_APP,SUM_ON_APP_DATE_PM,SUM_ON_APP_DATE,ACTIVITY_NAME_PM,ACTIVITY_NAME,LAST_TRANS,SUM_TRANS_NO_PM,SUM_TRANS_NO,SUM_TRANS_AMOUNT_PM,SUM_TRANS_AMOUNT,SUM_TRANS_DATE_PM,SUM_TRANS_DATE,SUM_TRANS_HOUR_PM,SUM_TRANS_HOUR,TRANS_LV1,TRANS_LV1_PM,TRANS_LV2,TRANS_LV2_PM
0,0,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 1, 0, 0]",11301.0,2.0,2019-09-01,0.0,74.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,[],[],NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[],[]
1,0,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 1, 0, 0]",11301.0,2.0,2019-10-01,1.0,44.0,2019-11-14,5.0,10.0,10.0,1.0,1.0,1.0,1.0,"[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",5.0,1.0,1.0,99800000.0,99800000.0,1.0,1.0,1.0,1.0,"[1, 0, 0]","[1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,0,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 1, 0, 0]",11301.0,2.0,2019-11-01,0.0,13.0,NaN,18.0,29.0,39.0,5.0,6.0,4.0,5.0,"[2, 4, 2, 2, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[3, 5, 3, 2, 5, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, ...",18.0,1.0,2.0,2000000.0,101800000.0,1.0,2.0,1.0,2.0,"[2, 0, 0]","[1, 0, 0]","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,0,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 1, 0, 0]",11301.0,2.0,2019-12-01,0.0,NaN,NaN,1.0,38.0,77.0,5.0,11.0,2.0,7.0,"[2, 4, 3, 2, 4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[5, 9, 6, 4, 9, 1, 0, 1, 4, 0, 0, 0, 0, 0, 0, ...",1.0,3.0,5.0,76000000.0,177800000.0,2.0,4.0,2.0,4.0,"[4, 0, 0]","[2, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,0,"[1, 0]","[1, 0]","[0, 1, 0, 0]","[1, 0]","[0, 1, 0, 0]",11301.0,2.0,2020-01-01,0.0,NaN,NaN,3.0,35.0,112.0,7.0,18.0,5.0,12.0,"[3, 7, 1, 0, 7, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, ...","[8, 16, 7, 4, 16, 1, 0, 1, 10, 0, 0, 0, 0, 0, ...",3.0,6.0,11.0,72019000.0,249819000.0,5.0,9.0,6.0,10.0,"[10, 0, 0]","[6, 0, 0]","[10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
